In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm import tqdm
import numpy as np
import time
from fake_useragent import UserAgent
import os
import pandas as pd
from unicodedata import normalize
import textract

Ezt az oldalt fogjuk scrapelni:
<br>
https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium&oldal=1

## URL linkek kiszedése 

## Tiltás elleni védelem

Usar Agent

In [2]:
ua = UserAgent()

In [3]:
ua.random

'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'

Proxik

In [4]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    soup = bs(response.content, 'lxml')
    table = soup.find('div', {'class': 'table-responsive'}).table.tbody
    table = table.find_all('tr')
    proxies = [(row.find_all('td')[0].text + ':' + row.find_all('td')[1].text) for row in table]
    print(f'Proxies: {len(proxies)}')
    return [{"http": f'http://{proxy}', "https": f'https://{proxy}'} for proxy in proxies]

In [5]:
proxies = get_proxies()

Proxies: 300


In [6]:
np.random.choice(proxies)

{'http': 'http://168.169.96.17:8080', 'https': 'https://168.169.96.17:8080'}

## Scrapelés

A letiltást megoldó részek

In [ ]:
session = requests.Session()

In [ ]:
session.proxies = np.random.choice(proxies)

session.headers = {'User-Agent' : ua.random}

session.timeout = 5

Küldjünk requestet

In [ ]:
result = session.get(f'{URL}1')

Készüljünk arra is, ha valami nem jó

In [ ]:
try:
    session = requests.Session()
    
    session.proxies = np.random.choice(proxies)
    
    session.headers = {'User-Agent' : ua.random}
    
    session.timeout = 5
    
    result = session.get(f'{URL}1')
except:
    print('HIBA')

Keressük meg az oldalszámokat

In [ ]:
# soup létrehozása
soup = bs(result.content, 'lxml')

# Oldalszám div
pages = soup.find('div', {'class': 'oldalszamok'})
print(pages.prettify())

Keressük meg a mostani oldalt. Ez akkor jó, ha megyegyezik azzal, amit az url-ben megadtunk.

In [ ]:
int(pages.find('span').text)

Ha jó az oldal, akkor nézzük meg a kérdéseket

In [ ]:
questions = soup.find_all('div', {'class': 'kerdeslista'})

# Hogy néz ki egy kérdés
print(questions[0].prettify())

Ne vegyük figyelembe a kiemeléseket

In [ ]:
questions = [q for q in questions if q.find('div', 'kerdes_kiemelt') == None]

Szedjük ki a linkeket

In [ ]:
links = [q.find('a')['href'] for q in questions]
links

In [ ]:
links = [re.sub('(.*)(/.*)', '\\1', URL) + link for link in links]
links

Az egész egy függvényben

In [ ]:
def get_urls(page_num):
    error = False
    last = False
    links = []
    
    ua = UserAgent()

    try:        
        session = requests.Session()

        session.proxies = np.random.choice(proxies)

        session.headers = {'User-Agent' : ua.random}

        session.timeout = 5

        result = session.get(f'{URL}{page_num}')
        
        soup = bs(result.content, 'lxml')
    
        time.sleep(np.random.uniform(5,10))
    except:
        error = True
        
    try:
        pages = soup.find('div', {'class': 'oldalszamok'})
        last = int(pages.find('span').text) != page_num
        if last:
            return error, last, []
    except:
        last = True
    
    try:
        questions = soup.find_all('div', {'class': 'kerdeslista'})

        questions = [q for q in questions if q.find('div', 'kerdes_kiemelt') == None]

        links = [q.find('a')['href'] for q in questions]

        links = [re.sub('(.*)(/.*)', '\\1', URL) + link for link in links]
    except:
        error = True

    return error, last, links

In [ ]:
get_urls(10)

Menjünk végig az oldalakon

In [ ]:
links = []
page_num = 1

while True:
    error, last, new_links = get_urls(page_num)
    print(f'page: {page_num}, error: {error}, last: {last}')
    
    if error == False:
        if last == True:
            break
        else:
            links.extend(new_links)
            page_num += 1
         

# kiírás csv-be
with open('links.txt', 'w') as file:
    file.write('\n'.join(links))

In [ ]:
links

# Oldalak mentése HTML-ben

In [8]:
session = requests.Session()

In [9]:
session.proxies = np.random.choice(proxies)

In [10]:
session.headers = {'User-Agent' : ua.random}

In [11]:
session.timeout = 5

In [12]:
with open('links.txt') as file:
    links = file.read().strip().split('\n')

In [13]:
result = session.get(links[3])

In [16]:
soup = bs(result.content, 'lxml')

# Letiltott az oldal?
soup.find('h2').text == 'Gyakorikerdesek.hu biztonsági kód'

False

Mappa létrehozása

In [17]:
if not os.path.exists('questions'):
    os.makedirs('questions')

Mentsük ki a HTML fájlt

In [ ]:
with open(os.path.join('questions', f'page_{1}.html'), 'wb') as file:
    file.write(result.content)

Nézzük meg az egyedi azaonosítókat

In [18]:
links[20]

'https://www.gyakorikerdesek.hu/kozoktatas-tanfolyamok__tanulasi-lehetosegek__10480176-aki-jelentkezett-iden-az-mcc-be-kapott-mar-emailt-hogy-felvettek-vagy-nem'

In [19]:
re.sub('(.*__)(\d+)(-.*)', '\\2', links[20])

'10480176'

Az egész egyben - addig fusson ameddig nem lesz jó

In [ ]:
def get_html_code(url):
    
    while True:
        try:
            ua = UserAgent()

            session = requests.Session()

            session.headers = {'User-Agent' : ua.random}
            
            session.proxies = np.random.choice(proxies)

            session.timeout = 5

            result = session.get(url)
            
            time.sleep(np.random.uniform(5,10))

            if not os.path.exists('questions'):
                os.makedirs('questions')
        
            unique_id = re.sub('(.*__)(\d+)(-.*)', '\\2', url)
            
            soup = bs(result.content, 'lxml')

            if soup.find('h2').text == 'Gyakorikerdesek.hu biztonsági kód':
                continue
            
            with open(os.path.join('questions', f'page_{unique_id}.html'), 'wb') as file:
                file.write(result.content)
                
            return
        except:
            pass

In [ ]:
get_html_code(links[1])

In [ ]:
for link in tqdm(links):
    get_html_code(link)

# Scraping one page from HTML code 

Nyissunk meg egy HTML file-t

In [20]:
os.listdir('questions')[0]

'page_7973530.html'

In [22]:
with open(os.path.join('questions', os.listdir('questions')[2]), 'rb') as file:
    response = file.read()
    
soup = bs(response, 'lxml')

Keressük meg a címet

In [23]:
def find_title(soup):
    return soup.find('div', {'class': 'kerdes_fejlec'}).find('h1').text

find_title(soup)

'Van reális esély, hogy felvegyenek az ELTE Bolyai\nszakkollégiumába elsőévesként?'

Na de milyen kódolás?

In [24]:
find_title(soup).encode()

b'Van re\xc3\xa1lis es\xc3\xa9ly, hogy felvegyenek az ELTE Bolyai\nszakkoll\xc3\xa9gium\xc3\xa1ba els\xc5\x91\xc3\xa9vesk\xc3\xa9nt?'

In [25]:
normalize('NFKD', find_title(soup))

'Van reális esély, hogy felvegyenek az ELTE Bolyai\nszakkollégiumába elsőévesként?'

In [26]:
normalize('NFKD', find_title(soup)).encode()

b'Van rea\xcc\x81lis ese\xcc\x81ly, hogy felvegyenek az ELTE Bolyai\nszakkolle\xcc\x81giuma\xcc\x81ba elso\xcc\x8be\xcc\x81veske\xcc\x81nt?'

In [32]:
def find_title(soup):
    title = soup.find('div', {'class': 'kerdes_fejlec'}).find('h1').text
    title = normalize('NFKD', title)
    return title
    
find_title(soup)

'Van reális esély, hogy felvegyenek az ELTE Bolyai\nszakkollégiumába elsőévesként?'

Keressük meg a tageket

In [33]:
def find_tags(soup):
    tag_div = soup.find('div', {'class': 'kerdes_kulcsszo'})
    tags = tag_div.find_all('span')
    tags = [tag.text.replace('#', '') for tag in tags]
    tags = [tag for tag in tags]
    tags = [normalize('NFKD', tag) for tag in tags]
    return tags

find_tags(soup)

['egyetem', 'matematika', 'fizika', 'felvételi', 'ELTE', 'szakkollégium']

Keressük meg a dátumot

In [34]:
def find_question_date(soup):
    date = soup.find('div', {'class': 'kerdes_statusz'}).text
    date = normalize('NFKD', date)
    return date

find_question_date(soup)

'2020. máj. 21. 11:39'

Keressük meg a kérdést

In [35]:
def find_question(soup):
    question_div = soup.find('div', {'class': 'kerdes_kerdes'})
    question = question_div.contents[1:]
    question = '\n'.join([ques if isinstance(ques, str) else ques.text for ques in question])
    question = normalize('NFKD', question)
    return question

find_question(soup)

'Kb. 220%-os érettségi átlagom lesz, határontúli vagyok. Fizika alapszakra jelentkezek, romániai versenyeredményeim vannak fizikából és matekból.\n'

Keressük meg a válaszokat

In [36]:
def find_answers(soup):
    return soup.find_all('div', {'class': ('valasz', 'kerdes')})[1:-1]

find_answers(soup)

[<div class="valasz" id="valasz-61876424"><div class="valasz_fejlec"> 1/7 <span class="anonim">anonim</span> <span class="vsz"><img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/><img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/><img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/><img alt="*" src="//static.gyakorikerdesek.hu/p/vsz0.png"/><img alt="*" src="//static.gyakorikerdesek.hu/p/vsz0.png"/></span> válasza:</div><div class="valasz_valasz"><div class="flex" style="flex-wrap: nowrap;"><div id="ertek61876424" style="margin-right: 10px;"><svg class="ertek" viewbox="0 0 100 100"><circle cx="50" cy="50" fill="#C4DCC5" r="45"></circle><path d="M50 10 a 40 40 0 0 1 0 80 a 40 40 0 0 1 0 -80" fill="none" stroke="#1F8D25" stroke-dasharray="226.1,25.1" stroke-linecap="round" stroke-width="7"></path><text dy="7" font-size="20" text-anchor="middle" x="50" y="50">90%</text></svg></div><div id="valasz61876424" style="min-width: 680px;"><p style="margin: 0 0 7px 0;">"Kb. 220

Keressük meg a válaszok dátumát

In [37]:
answer = find_answers(soup)

In [38]:
def find_answer_date(answer):
    date_div = answer.find('div', {'class': ('valasz_statusz', 'kerdes_statusz')})
    date = date_div.contents[0].text
    date = normalize('NFKD', date)
    return date

find_answer_date(find_answers(soup)[3])

'2020. máj. 21. 12:43'

Keressük meg a válaszok szövegét

In [39]:
def find_answer_text(answer):
    answer_div = answer.find('div', {'class': ('valasz_valasz', 'kerdes_kerdes')})
    answer = answer_div.findChild().contents[-1].contents
    answer = '\n'.join([ans if isinstance(ans, str) else ans.text for ans in answer])
    answer = normalize('NFKD', answer)
    return answer

find_answer_text(find_answers(soup)[6])

'Annyira senki nem makacs, hogy ne vegyenek föl, mert nincs magyarországi emelt érettségid :)\n(Pláne a mostani rémtörténetek mellett a határőrökkel:  [link]  )\n\nNagyobb kérdés sajnos az, hogy egyáltalán megnyithatnak-e a kollégiumok rendesen ősszel... :('

minden egyben

In [40]:
def extract_content(soup, unique_id):
    data = []
    
    title = find_title(soup)
    tags = find_tags(soup)
    date = find_question_date(soup)
    text = find_question(soup)
    
    question_data = {'title': title,
                     'tag': tags,
                     'date': date,
                     'text': text,
                     'is_question': True,
                     'id': unique_id}
    
    data.append(question_data)
    
    answers = find_answers(soup)

    for answer in answers:
        
        date = find_answer_date(answer)
        text = find_answer_text(answer)
        
        answer_data = {'title': title,
                       'tag': tags,
                       'date': date,
                       'text': text,
                       'is_question': False,
                       'id': unique_id}
        
        data.append(answer_data)
    
    return data

extract_content(soup, 'barmi')

[{'title': 'Van reális esély, hogy felvegyenek az ELTE Bolyai\nszakkollégiumába elsőévesként?',
  'tag': ['egyetem',
   'matematika',
   'fizika',
   'felvételi',
   'ELTE',
   'szakkollégium'],
  'date': '2020. máj. 21. 11:39',
  'text': 'Kb. 220%-os érettségi átlagom lesz, határontúli vagyok. Fizika alapszakra jelentkezek, romániai versenyeredményeim vannak fizikából és matekból.\n',
  'is_question': True,
  'id': 'barmi'},
 {'title': 'Van reális esély, hogy felvegyenek az ELTE Bolyai\nszakkollégiumába elsőévesként?',
  'tag': ['egyetem',
   'matematika',
   'fizika',
   'felvételi',
   'ELTE',
   'szakkollégium'],
  'date': '2020. máj. 21. 11:46',
  'text': '"Kb. 220%-os érettségi átlagom lesz"\nEz alatt mit értesz? román vagy magyar érettségi? egyáltalán hogy lehet 100 fölött kapni?\n\n\n"Fizika alapszakra jelentkezek, romániai versenyeredményeim vannak fizikából és matekból." Milyen versenyeredmények? Románia jelentősen jobb gimib

Menjünk végig az összes file-on

In [41]:
data = []

for doc in tqdm(os.listdir(os.path.join('questions'))):
    print(doc)
    
    unique_id = re.sub('page_(\d+)\.html', '\\1', doc)

    with open(os.path.join('questions', doc), 'rb') as file:
        response = file.read()

    soup = bs(response, 'lxml')

    file_data = extract_content(soup, unique_id)

    data.extend(file_data)

  2%|▏         | 4/191 [00:00<00:05, 36.17it/s]

page_7973530.html
page_8000830.html
page_10495238.html
page_9751318.html
page_7059731.html
page_10468461.html
page_10908121.html
page_7998008.html
page_10618580.html


  4%|▍         | 8/191 [00:00<00:04, 38.75it/s]


AttributeError: 'NoneType' object has no attribute 'find_all'

Készüljünk ismét a rosszra

In [42]:
def extract_content(soup, unique_id):
    data = []
    
    try:
        title = find_title(soup)
    except:
        title = None
        
    try:
        tags = find_tags(soup)
    except:
        tags = None
        
    try:
        date = find_question_date(soup)
    except:
        date = None
        
    try:
        text = find_question(soup)
    except:
        text = None
    
    question_data = {'title': title,
                     'tag': tags,
                     'date': date,
                     'text': text,
                     'is_question': True,
                     'id': unique_id}
    
    data.append(question_data)
    
    try:
        answers = find_answers(soup)
    except:
        answers = []
        
    for answer in answers:
        
        try:
            date = find_answer_date(answer)
        except:
            date = None
            
        try:
            text = find_answer_text(answer)
        except:
            text = None
        
        answer_data = {'title': None,
                       'tag': tags,
                       'date': date,
                       'text': text,
                       'is_question': False,
                       'id': unique_id}
        
        data.append(answer_data)
    
    return data

extract_content(soup, 'barmi')

[{'title': 'Szakkollégiumba való felvételhez önéletrajzot kérnek.\nMit lehetne beleírni?',
  'tag': None,
  'date': '2020. júl. 29. 20:10',
  'text': '',
  'is_question': True,
  'id': 'barmi'},
 {'title': None,
  'tag': None,
  'date': '2020. júl. 29. 22:08',
  'text': 'Személyes adatok, iskolák, nyelvvizsgàk, versenyeredmények, konferenciák, demonstrátorság, diákköri tagság pl. Ilyen jellegű dolgokra kíváncsiak.',
  'is_question': False,
  'id': 'barmi'}]

Menjünk végig ismét

In [43]:
data = []

for doc in tqdm(os.listdir(os.path.join('questions'))):
    
    unique_id = re.sub('page_(\d+)\.html', '\\1', doc)

    with open(os.path.join('questions', doc), 'rb') as file:
        response = file.read()

    soup = bs(response, 'lxml')

    file_data = extract_content(soup, unique_id)

    data.extend(file_data)

100%|██████████| 191/191 [00:03<00:00, 54.03it/s]


Átláthatóbban dataframe-ben

In [44]:
df = pd.DataFrame(data)
df

,title,tag,date,text,is_question,id
0,Az ELTE Bolyai szakkollégiumába hogyan zajli...,"[egyetem, ELTE, IK, Bolyai, szakkollégium, sz...",2016. júl. 26. 16:47,Illetve mi az a minimum versenyeredmények/tan...,True,7973530
1,"Tudtok olyan szakkolégiumot Budapesten, ahova...","[egyetem, Budapest, közösség, szakkollégiu...",2016. aug. 5. 20:38,,True,8000830
2,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. aug. 22. 22:48,Váci Mihály kolesz,False,8000830
3,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. szept. 7. 22:01,Na sikerült vagy mi lett végül?,False,8000830
4,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. szept. 7. 22:05,Szerintem a legtobb szakkolinal nincs megkotes...,False,8000830
...,...,...,...,...,...,...
541,None,"[kollégium, PPKE, Piliscsaba, Iosephinum Szak...",2020. máj. 14. 12:33,"Köszi, hát még átgondolom ezt, maga a koll...",False,10449407
542,None,"[kollégium, PPKE, Piliscsaba, Iosephinum Szak...",2020. máj. 14. 14:40,Nem vészes ez az ingázás. Ha valóban tanul...,False,10449407
543,Szegeden a Móra Ferenc kollégium ugyanaz min...,None,2016. aug. 10. 17:47,vagy 2 különböző koli? hol található a s...,True,8012920
544,None,None,2016. aug. 10. 17:47,mennyi az esély bekerülni?,False,8012920


Javítsuk ki a szöveget

In [45]:
df['text'] = df['text'].apply(lambda x: x if x != '' else None)
df

,title,tag,date,text,is_question,id
0,Az ELTE Bolyai szakkollégiumába hogyan zajli...,"[egyetem, ELTE, IK, Bolyai, szakkollégium, sz...",2016. júl. 26. 16:47,Illetve mi az a minimum versenyeredmények/tan...,True,7973530
1,"Tudtok olyan szakkolégiumot Budapesten, ahova...","[egyetem, Budapest, közösség, szakkollégiu...",2016. aug. 5. 20:38,None,True,8000830
2,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. aug. 22. 22:48,Váci Mihály kolesz,False,8000830
3,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. szept. 7. 22:01,Na sikerült vagy mi lett végül?,False,8000830
4,None,"[egyetem, Budapest, közösség, szakkollégiu...",2016. szept. 7. 22:05,Szerintem a legtobb szakkolinal nincs megkotes...,False,8000830
...,...,...,...,...,...,...
541,None,"[kollégium, PPKE, Piliscsaba, Iosephinum Szak...",2020. máj. 14. 12:33,"Köszi, hát még átgondolom ezt, maga a koll...",False,10449407
542,None,"[kollégium, PPKE, Piliscsaba, Iosephinum Szak...",2020. máj. 14. 14:40,Nem vészes ez az ingázás. Ha valóban tanul...,False,10449407
543,Szegeden a Móra Ferenc kollégium ugyanaz min...,None,2016. aug. 10. 17:47,vagy 2 különböző koli? hol található a s...,True,8012920
544,None,None,2016. aug. 10. 17:47,mennyi az esély bekerülni?,False,8012920


Mentsük ki az egészet egy csv fileba

In [ ]:
df.to_csv('data.csv', sep = ';', encoding = 'utf-8')

# Adatszerzés PDF-ből

Szöveg kiszedése

In [70]:
text = textract.process('rajk_modell.pdf', method='pdftotext')

Nézzünk bele

In [71]:
text[:3000]

b'CHIK \xc3\x81N ATTI LA \xe2\x80\x93 I LY \xc3\x89S M\xc3\x81RTON\n\nA Rajk-modell: egy pedag\xc3\xb3giai\nk\xc3\xads\xc3\xa9rlet \xc3\xa9s tapasztalatai\n\nBevezet\xc3\xa9s\n\nE\n\ngy orsz\xc3\xa1g oktat\xc3\xa1si rendszer\xc3\xa9nek sz\xc3\xadnvonal\xc3\xa1t egyebek mellett a l\xc3\xa9tez\xc5\x91 form\xc3\xa1k v\xc3\xa1ltozatoss\xc3\xa1ga, az innovat\xc3\xadv k\xc3\xads\xc3\xa9rletek befogad\xc3\xa1sa is m\xc3\xa9lyen jellemzi. A szakkoll\xc3\xa9giumok val\xc3\xb3ban innovat\xc3\xadv form\xc3\xa1k: b\xc3\xa1r term\xc3\xa9szetesen m\xc3\xa1s orsz\xc3\xa1gokban is tal\xc3\xa1lhat\xc3\xb3k\nolyan int\xc3\xa9zm\xc3\xa9nyek, amelyek a hazai szakkoll\xc3\xa9giumok h\xc3\xa1rom mark\xc3\xa1ns jellemz\xc5\x91j\xc3\xa9nek, a\nszakmais\xc3\xa1gnak, k\xc3\xb6z\xc3\xb6ss\xc3\xa9gis\xc3\xa9gnek \xc3\xa9s t\xc3\xa1rsadalmi \xc3\xa9rz\xc3\xa9kenys\xc3\xa9gnek ak\xc3\xa1r t\xc3\xb6bb von\xc3\xa1s\xc3\xa1val is\nrendelkeznek, tapasztalataink szerint ezek integr\xc3\xa1lt egys\xc3\xa9g\xc3\xa9nek meg

Alakítsuk át

In [72]:
text = text.decode()
text[:3000]

'CHIK ÁN ATTI LA – I LY ÉS MÁRTON\n\nA Rajk-modell: egy pedagógiai\nkísérlet és tapasztalatai\n\nBevezetés\n\nE\n\ngy ország oktatási rendszerének színvonalát egyebek mellett a létező formák változatossága, az innovatív kísérletek befogadása is mélyen jellemzi. A szakkollégiumok valóban innovatív formák: bár természetesen más országokban is találhatók\nolyan intézmények, amelyek a hazai szakkollégiumok három markáns jellemzőjének, a\nszakmaiságnak, közösségiségnek és társadalmi érzékenységnek akár több vonásával is\nrendelkeznek, tapasztalataink szerint ezek integrált egységének megvalósítása hungaricum. Külön érdekességet és fontosságot ad a magyar szakkollégiumi rendszernek, hogy\neredeti formájában alulról jövő kezdeményezésként jött létre, s bár évtizedek óta elismerten bizonyít, mind a mai napig lényegében javarészt állami támogatás nélkül működik.\nLehet persze, hogy ez hasznára vált: a hazai felsőoktatás-irányítás több évtizedes cikkcakkjainak kényszerű követése valószínűleg ink

Normalizáljuk

In [73]:
text = normalize('NFKD', text)
text[:3000]

'CHIK ÁN ATTI LA – I LY ÉS MÁRTON\n\nA Rajk-modell: egy pedagógiai\nkísérlet és tapasztalatai\n\nBevezetés\n\nE\n\ngy ország oktatási rendszerének színvonalát egyebek mellett a létező formák változatossága, az innovatív kísérletek befogadása is mélyen jellemzi. A szakkollégiumok valóban innovatív formák: bár természetesen más országokban is találhatók\nolyan intézmények, amelyek a hazai szakkollégiumok három markáns jellemzőjének, a\nszakmaiságnak, közösségiségnek és társadalmi érzékenységnek akár több vonásával is\nrendelkeznek, tapasztalataink szerint ezek integrált egységének megvalósítása hungaricum. Külön érdekességet és fontosságot ad a magyar szakkollégiumi rendszernek, hogy\neredeti formájában alulról jövő kezdeményezésként jött létre, s bár évtizedek óta elismerten bizonyít, mind a mai napig lényegében javarészt állami támogatás nélkül működik.\nLehet persze, hogy ez hasznára vált: a h